# <u><b>Import Libraries:-</u>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# <u><b>Load the Data:-</u>

In [2]:
a = r'C:\Python\Project -4\Edulyt India - Data for Project - 4.xlsx' 
New_Account = pd.read_excel(a, sheet_name='New_Account') 
New_Card = pd.read_excel(a, sheet_name='New_Card') 
District = pd.read_excel(a, sheet_name='District')
Loan = pd.read_excel(a, sheet_name='Loan')
New_Client = pd.read_excel(a, sheet_name='New_Client')
New_Disposition = pd.read_excel(a, sheet_name='New_Disposition')
New_Transaction = pd.read_excel(a, sheet_name='New_Transaction')
Order = pd.read_excel(a, sheet_name='Order')

# <u><b>Tasks:-<u>

## 1) Task 1:-

In [16]:
# Task 1: Prepare the Aggregated View of the transactions basis on 2 things i.e.
# a. Account Wise
# b. Month Wise
# c. Account and Month wise

District_clean = District[pd.to_numeric(District['A1'], errors='coerce').notnull()].copy()
District_clean['A1'] = District_clean['A1'].astype(int)

account_district = New_Account.merge(District_clean, left_on='district_id', right_on='A1', how='left')

moravia_prague = account_district[account_district['A2'].isin(['Moravia', 'Prague'])]

credit_tx = New_Transaction[
    (New_Transaction['account_id'].isin(moravia_prague['account_id'])) &
    (New_Transaction['type'].str.lower() == 'credit')
].copy()

credit_tx['month'] = pd.to_datetime(credit_tx['date'], errors='coerce').dt.to_period('M')



In [7]:

account_summary = credit_tx.groupby('account_id')['amount'].sum().reset_index()

account_summary

,account_id,amount


In [8]:

month_summary = credit_tx.groupby('month')['amount'].sum().reset_index()

month_summary 

,month,amount


In [9]:

account_month_summary = credit_tx.groupby(['account_id', 'month'])['amount'].sum().reset_index()
account_month_summary

,account_id,month,amount


## 2)  Task 2:-

In [35]:
# Task 2: Analysis on Highly populated versus Low populated districts.

District_clean = District[pd.to_numeric(District['A1'], errors='coerce').notnull()].copy()
District_clean['A1'] = District_clean['A1'].astype(int)
account_district = New_Account.merge(District_clean, left_on='district_id', right_on='A1', how='left')
New_Transaction['date'] = pd.to_datetime(New_Transaction['date'], errors='coerce')
last_3_months = New_Transaction['date'].max() - pd.DateOffset(months=3)
tx_region = New_Transaction.merge(New_Account[['account_id', 'district_id']], on='account_id', how='left')
tx_region = tx_region.merge(District_clean, left_on='district_id', right_on='A1', how='left')
tx_region_recent = tx_region[tx_region['date'] >= last_3_months]
pop_sorted = District_clean.sort_values('A3', ascending=False) 
high_pop = pop_sorted.head(5)['A1']
low_pop = pop_sorted.tail(5)['A1']



In [19]:
high_tx = tx_region_recent[tx_region_recent['district_id'].isin(high_pop)]
high_tx

,Unnamed: 0,account_id,date,type,operation,amount,balance,k_symbol,district_id,A1,...,A7,A8,A9,A10,A11,A12,A13,A14,A15,A16
35,36,3871,1970-01-01 00:00:00.000930108,CREDIT,CREDIT IN CASH,300.0,300.0,NaN,26,26,...,0,1,1,100.0,10787,2.64,3.09,131,6041,6261
50,51,1979,1970-01-01 00:00:00.000930111,CREDIT,CREDIT IN CASH,900.0,900.0,NaN,26,26,...,0,1,1,100.0,10787,2.64,3.09,131,6041,6261
89,90,212,1970-01-01 00:00:00.000930120,CREDIT,CREDIT IN CASH,200.0,200.0,NaN,27,27,...,10,0,7,43.8,8561,0.65,1.29,110,1029,1127
103,104,280,1970-01-01 00:00:00.000930123,CREDIT,CREDIT IN CASH,300.0,300.0,NaN,25,25,...,3,2,10,61.7,8554,2.47,2.68,113,1822,2218
105,106,4055,1970-01-01 00:00:00.000930124,CREDIT,CREDIT IN CASH,200.0,200.0,NaN,23,23,...,4,3,9,85.5,8624,1.79,2.66,102,2879,3198
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1048355,1048356,3387,1970-01-01 00:00:00.000981218,CREDIT,CREDIT IN CASH,1100.0,80415.9,NaN,25,25,...,3,2,10,61.7,8554,2.47,2.68,113,1822,2218
1048384,1048385,553,1970-01-01 00:00:00.000981218,CREDIT,CREDIT IN CASH,800.0,42543.4,NaN,26,26,...,0,1,1,100.0,10787,2.64,3.09,131,6041,6261
1048385,1048386,554,1970-01-01 00:00:00.000981218,CREDIT,CREDIT IN CASH,1600.0,25251.7,NaN,26,26,...,0,1,1,100.0,10787,2.64,3.09,131,6041,6261
1048472,1048473,3988,1970-01-01 00:00:00.000981218,CREDIT,CREDIT IN CASH,1000.0,37624.1,NaN,23,23,...,4,3,9,85.5,8624,1.79,2.66,102,2879,3198


In [20]:
low_tx = tx_region_recent[tx_region_recent['district_id'].isin(low_pop)]
low_tx

,Unnamed: 0,account_id,date,type,operation,amount,balance,k_symbol,district_id,A1,...,A7,A8,A9,A10,A11,A12,A13,A14,A15,A16
6,7,1539,1970-01-01 00:00:00.000930103,CREDIT,CREDIT IN CASH,600.0,600.0,NaN,1,1,...,0,1,1,100.0,12541,0.29,0.43,167,85677,99107
25,26,2393,1970-01-01 00:00:00.000930107,CREDIT,CREDIT IN CASH,900.0,900.0,NaN,10,10,...,4,3,5,46.7,10124,0.56,0.54,141,3810,4316
31,32,1637,1970-01-01 00:00:00.000930108,CREDIT,CREDIT IN CASH,500.0,500.0,NaN,1,1,...,0,1,1,100.0,12541,0.29,0.43,167,85677,99107
33,34,764,1970-01-01 00:00:00.000930108,CREDIT,CREDIT IN CASH,1100.0,1100.0,NaN,1,1,...,0,1,1,100.0,12541,0.29,0.43,167,85677,99107
40,41,1730,1970-01-01 00:00:00.000930109,CREDIT,CREDIT IN CASH,900.0,900.0,NaN,1,1,...,0,1,1,100.0,12541,0.29,0.43,167,85677,99107
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1048549,1048550,7745,1970-01-01 00:00:00.000981219,CREDIT,CREDIT IN CASH,4500.0,25398.7,NaN,2,2,...,6,2,5,46.7,8507,1.67,1.85,132,2159,2674
1048554,1048555,3282,1970-01-01 00:00:00.000981219,DEBIT,CASH WITHDRAWAL,400.0,17225.5,NaN,1,1,...,0,1,1,100.0,12541,0.29,0.43,167,85677,99107
1048560,1048561,90,1970-01-01 00:00:00.000981219,DEBIT,CASH WITHDRAWAL,16600.0,27870.4,NaN,1,1,...,0,1,1,100.0,12541,0.29,0.43,167,85677,99107
1048568,1048569,1890,1970-01-01 00:00:00.000981219,DEBIT,CASH WITHDRAWAL,14849.0,10268.2,NaN,1,1,...,0,1,1,100.0,12541,0.29,0.43,167,85677,99107


In [21]:
high_summary = high_tx.groupby('type')['amount'].sum().reset_index(name='high_pop_total')

high_summary

,type,high_pop_total
0,CREDIT,135147716.6
1,DEBIT,125603581.4


In [22]:
low_summary = low_tx.groupby('type')['amount'].sum().reset_index(name='low_pop_total')
low_summary

,type,low_pop_total
0,CREDIT,546154044.1
1,DEBIT,513162916.4


## 3) Task 3:-

In [23]:
# Task 3: How many cards are issued to mid age females

disp_client = New_Disposition.merge(New_Client, on='client_id')
disp_card = New_Card.merge(disp_client, on='disp_id')
disp_card['birth_year'] = pd.to_datetime(disp_card['birth_number']).dt.year
disp_card['age'] = 2025 - disp_card['birth_year']
mid_age_female_cards = disp_card[(disp_card['age'] >= 30) & (disp_card['age'] <= 50) & (disp_card['gender'] == 'F')]
len(mid_age_female_cards)

0

## 4) Task 4:-

In [24]:
# Task 4: Number of cards issued in district where average salary is more than 9000, is it a good strategy?

acc_disp = New_Account.merge(New_Disposition, on='account_id')
acc_disp_dist = acc_disp.merge(District_clean[['A1', 'A4']], left_on='district_id', right_on='A1')
high_salary_disp = acc_disp_dist[acc_disp_dist['A4'] > 9000]
high_salary_cards = New_Card[New_Card['disp_id'].isin(high_salary_disp['disp_id'])]
len(high_salary_cards)


892

## Task 5:- 

In [25]:
# Task 5: Loans in districts where crimes for code 95 > 6000

loan_district = Loan.merge(New_Account, on='account_id')
loan_district = loan_district.merge(District_clean, left_on='district_id', right_on='A1')
crime95_loans = loan_district[loan_district['A11'] > 6000]
loan_count_per_district = crime95_loans.groupby('A2')['loan_id'].count()

loan_count_per_district

A2
Benesov              6
Beroun               6
Blansko              7
Breclav              7
Brno - mesto        24
                    ..
Vsetin               6
Vyskov               7
Zdar nad Sazavou     7
Zlin                17
Znojmo               6
Name: loan_id, Length: 77, dtype: int64

## Task 6:-

In [26]:
# Task 6: Money collected from other banks (if unemployment rate > 2%)

high_unemp_districts = District_clean[District_clean['A10'] > 2]['A1']
order_merge = Order.merge(New_Account, on='account_id')
order_merge = order_merge[order_merge['bank_to'] != 'BANK']  # exclude self bank
order_merge = order_merge[order_merge['account_id'].isin(New_Account[New_Account['district_id'].isin(high_unemp_districts)]['account_id'])]
total_amount_from_others = order_merge['amount'].sum()
total_amount_from_others

np.float64(21228993.6)

## Task 7:-

In [29]:
# Task 7: Customer profiles based on max money paid to categories

tx_cat = New_Transaction.groupby(['account_id', 'k_symbol'])['amount'].sum().reset_index()
tx_cat = tx_cat.merge(New_Account[['account_id', 'district_id']], on='account_id')
tx_cat = tx_cat.merge(District_clean[['A1', 'A2']], left_on='district_id', right_on='A1')
profile_insurance = tx_cat[tx_cat['k_symbol'] == 'insurance'].groupby('A2')['amount'].sum().sort_values(ascending=False)
profile_household = tx_cat[tx_cat['k_symbol'] == 'household'].groupby('A2')['amount'].sum().sort_values(ascending=False)
profile_leasing = tx_cat[tx_cat['k_symbol'] == 'leasing'].groupby('A2')['amount'].sum().sort_values(ascending=False)
profile_loan = tx_cat[tx_cat['k_symbol'] == 'loan payment'].groupby('A2')['amount'].sum().sort_values(ascending=False)

print("\n✅ Task 7: Top paying districts per category")
print(profile_insurance)
print(profile_household)
print(profile_leasing)
print(profile_loan)


✅ Task 7: Top paying districts per category
Series([], Name: amount, dtype: float64)
Series([], Name: amount, dtype: float64)
Series([], Name: amount, dtype: float64)
Series([], Name: amount, dtype: float64)


## Task 8:-

In [30]:
# Task 8: Customer profile based on loan status (paid/not)

loan_status = Loan.merge(New_Account, on='account_id').merge(District_clean, left_on='district_id', right_on='A1')
loan_profile = loan_status.groupby(['A2', 'status'])['loan_id'].count().unstack(fill_value=0)
loan_profile.head()

status,A,B,C,D
A2,,,,
Benesov,1,0,5,0
Beroun,1,0,3,2
Blansko,3,0,3,1
Breclav,1,1,5,0
Brno - mesto,4,2,15,3


## Task 9:-

In [31]:
# Task 9: Relate loan status with Crime, Unemployment, Salary

loan_district_stats = Loan.merge(New_Account, on='account_id').merge(District_clean, left_on='district_id', right_on='A1')
loan_status_stats = loan_district_stats.groupby('status')[['A11', 'A10', 'A4']].mean().rename(
    columns={'A11': 'avg_crime', 'A10': 'avg_unemployment', 'A4': 'avg_salary'}
)

loan_status_stats

,avg_crime,avg_unemployment,avg_salary
status,,,
A,9685.679803,69.427586,316102.285714
B,9432.129032,68.470968,246433.419355
C,9435.290323,67.596526,254912.947891
D,9333.911111,68.075556,244477.933333


## Task 10:-

In [32]:
# Task 10: Districts where permanent orders + loans exist

perm_order = Order[Order['k_symbol'] == 'permanent order']
perm_accounts = perm_order['account_id'].unique()
loan_accounts = Loan['account_id'].unique()
common_accounts = set(perm_accounts).intersection(set(loan_accounts))
common_acc_info = New_Account[New_Account['account_id'].isin(common_accounts)]
districts_with_both = common_acc_info.merge(District_clean, left_on='district_id', right_on='A1')['A2'].value_counts()

districts_with_both

Series([], Name: count, dtype: int64)

## Task 12:-

In [33]:
# Task 12: How many customers having credit card are also availing the loan facilities.

disp_card = New_Card.merge(New_Disposition, on='disp_id')
card_accounts = disp_card['account_id'].unique()
loan_accounts = Loan['account_id'].unique()
card_loan_overlap = set(card_accounts).intersection(set(loan_accounts))

len(card_loan_overlap)


170

## Task 14:-

In [34]:
# Tak 14: How will you analyze the performance of Mid age vs adults in terms of loan repayments.

New_Client['birth_year'] = New_Client['birth_number'].astype(str).str[:2].astype(int)
New_Client['birth_year'] = New_Client['birth_year'].apply(lambda x: 1900 + x if x > 24 else 2000 + x)
New_Client['age'] = 2025 - New_Client['birth_year']
def age_group(age):
    if 30 <= age <= 50:
        return 'mid-age'
    elif age > 50:
        return 'adult'
    else:
        return 'young'

New_Client['group'] = New_Client['age'].apply(age_group)
client_loan = New_Client.merge(New_Disposition, on='client_id')
client_loan = client_loan.merge(New_Account[['account_id']], on='account_id')
client_loan = client_loan.merge(Loan[['account_id', 'status']], on='account_id')
loan_by_group = client_loan.groupby(['group', 'status'])['account_id'].count().unstack(fill_value=0)

loan_by_group

status,A,B,C,D
group,,,,
adult,223,26,433,38
mid-age,35,5,60,7
